In [ ]:
import urllib3
import json
import pandas as pd
import numpy  as np
import time
import datetime

In [ ]:
# 検索クエリ
def req_params(start, end):
    params = {'query'        : '(btc OR bitcoin) -giveaway lang:en -is:retweet -is:reply -is:nullcast',
              'start_time'   : '2021-' + start + '-01T00:00:00Z',
              'end_time'     : '2021-' + end   + '-01T00:00:00Z',
              'max_results'  : 500,
              'expansions'   : 'author_id',
              'tweet.fields' : 'text,created_at,public_metrics',
              'user.fields'  : 'name,username,public_metrics'
              }
    
    return params

In [ ]:
# http,key
config = pd.read_csv("./config_academic.csv")
twitterAPIBearerToken=config['twitterApiBearerToken'][0]

http = urllib3.PoolManager()
key  = twitterAPIBearerToken

In [ ]:
# ツイート収集リクエスト
def req_tweet(http, key, searchFeild):
    url  = 'https://api.twitter.com/2/tweets/search/all'

    req = http.request('GET',
                        url,
                        headers= {'Authorization': 'Bearer '+key},
                        fields = searchFeild
                      )

    result = json.loads(req.data)
    if (req.status != 200):
        print('検索失敗')
        print(req.status)
        print(result['errors'])
        
    return result

In [ ]:
#ツイート収集実行
def get_tweet(params):
    next_token    = ''
    break_flag    = False
    request_count = 0
    tweet_count   = 0
    counter       = 1
    t_data        = []  # tweetのリスト
    u_data        = []  # userのリスト

    # 次ページがなくなるまで次ページのクエリを取得
    while True:
        try:
            data['meta']['next_token']

        # n+1回目：次ページがない(next_tokenがない)のでループを抜ける   
        except KeyError:
            break_flag = True

        # 1回目 
        except NameError:
            query = params

        ## 2≦k≦n回目　
        else:
            next_token = data['meta']['next_token']
            params['next_token'] = next_token
            query = params

        finally:
            if break_flag == True:
                break

            # 1リクエストで収集したデータ
            data = req_tweet(http, key, query)

            # tweet.fields
            if ('data' in data):          
                for i in range(data['meta']['result_count']):
                    t_data.append([data['data'][i]['text'],
                                   data['data'][i]['public_metrics']['like_count'],
                                   data['data'][i]['public_metrics']['retweet_count'],
                                   data['data'][i]['created_at'],
                                   data['data'][i]['author_id']])
                    i += 1

            # user.fields
            if ('includes' in data):
                for user in data['includes']['users']:
                    u_data.append([user['name'],
                                   '@'+user['username'],
                                   user['public_metrics']['following_count'],
                                   user['public_metrics']['followers_count'],
                                   user['id']])

            tweet_count += data['meta']['result_count']
            request_count += 1
            if request_count >= 300: # 300requestを超えたら止める
                dt_now = datetime.datetime.now()
                print(dt_now)
                print('{}回目. 300リクエストを超えるため、15分間停止します...'.format(counter))
                time.sleep(15.01*60) # 15分間（余裕をみてプラス1秒弱）中断
                dt_now = datetime.datetime.now()
                print(dt_now)
                print('15分経過、収集を再開します')
                counter += 1
                request_count = 0
                
    return t_data,u_data,tweet_count

In [ ]:
# csv出力
def to_csv(i,start_time, end_time, t_data, u_data, tweet_count):
    print("2021年{}月のツイート{}件収集完了".format(i,tweet_count))

    # 収集ログ
    dt_now = datetime.datetime.now()
    df_log = pd.read_csv('./tweet-of-btc/assembling-log.csv')
    s = pd.Series([dt_now, start_time, end_time, tweet_count], index=['収集時刻', '区間(start)','区間(end)','収集件数'])
    df_log = df_log.append(s, ignore_index=True)
    df_log.to_csv('./tweet-of-btc/assembling-log.csv',index=False)

    # t_data, u_data
    df1 = pd.DataFrame(data=t_data, columns=['Tweet','Favorite','RT','created_at','author_id'])
    df1.to_csv(f'./tweet-of-btc/2021-0{i}_tlist.csv', index=False)
    df2 = pd.DataFrame(data=u_data, columns=['Name','User_Name','Following','Followers','author_id'])
    df2.to_csv(f'./tweet-of-btc/2021-0{i}_ulist.csv', index=False)

    # t_data × u_data
#    df3 = pd.merge(df2 ,df1, on='author_id')

    # ツイート時刻順にソート
#   df3['created_at'] = pd.to_datetime(df3['created_at'], infer_datetime_format= True)
#   sorted_df = df3.sort_values(by = 'created_at', ascending = True, inplace = True) 
#   df3.to_csv('./tweet-of-btc/2021-0{i}.csv', index=False)

In [ ]:
# main
dt_now = datetime.datetime.now()
print(dt_now)
print('収集を開始します')
start = '0' + str(1)
end   = '0' + str(2)
params = req_params(start, end)
t_data, u_data, tweet_count = get_tweet(params)
to_csv(1,params['start_time'],params['end_time'],t_data, u_data, tweet_count)